In [ ]:
import os
import numpy as np

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Dropout, Flatten, MaxPool2D
from keras import optimizers
from keras.callbacks import ReduceLROnPlateau

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Data Preparation

In [ ]:
# Data are in dataset folder in zipped format
!ls "drive/My Drive/ENGR635-Deep Learning System Design Project/Dataset/fer2013/"

In [ ]:
!ls 

In [ ]:
# This creates a temporary folder in drive root folder, so it will have to be reloaded again when required after terminating the session
# Permanent data are stored in dataset in zipped format
# This copies the zipped file and store in root of google drive temporarily 
# This allows colab to run faster by taking data from root folder, rather than from actual drive location.
! rm -rf Training; mkdir Training
! unzip -q "drive/My Drive/ENGR635-Deep Learning System Design Project/Dataset/fer2013/Training.zip" -d Training

! rm -rf Validation; mkdir Validation
! unzip -q "drive/My Drive/ENGR635-Deep Learning System Design Project/Dataset/fer2013/PublicTest.zip" -d Validation

! rm -rf Test; mkdir Test
! unzip -q "drive/My Drive/ENGR635-Deep Learning System Design Project/Dataset/fer2013/PrivateTest.zip" -d Test

In [ ]:
!ls

In [ ]:
!ls -l Training/

In [ ]:
%%bash
root='Training/'
IFS=$(echo -en "\n\b")
(for dir in $(ls -1 "$root")
    do printf "$dir: " && ls -i "$root$dir" | wc -l
 done)

In [ ]:
%%bash
root='Validation/'
IFS=$(echo -en "\n\b")
(for dir in $(ls -1 "$root")
    do printf "$dir: " && ls -i "$root$dir" | wc -l
 done)

In [ ]:
%%bash
root='Test/'
IFS=$(echo -en "\n\b")
(for dir in $(ls -1 "$root")
    do printf "$dir: " && ls -i "$root$dir" | wc -l
 done)

In [ ]:
# Setting up path for training, validation and test directory
train_dir = "Training/"
validation_dir = "Validation/"
test_dir = "Test/"

## All data are ready

### Setting up image generator with data augmentation

In [ ]:
# Image height and width initialization
img_height = 224
img_width = 224

In [ ]:
# Image Data Generator setup
batch_size = 50
train_datagen = ImageDataGenerator(rescale=1./255,
                                   featurewise_center=False,
                                   featurewise_std_normalization=False,
                                   rotation_range=30,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   zoom_range=0.1,
                                   horizontal_flip=True
                                   )
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (img_height, img_width),
    batch_size = batch_size,
    shuffle = True,
    class_mode='categorical'
)
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size = (img_height, img_width),
    batch_size = batch_size,
    class_mode = 'categorical'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size = (img_height, img_width),
    batch_size = batch_size,
    class_mode = 'categorical'
)

In [ ]:
print(train_generator.class_indices)
print(validation_generator.class_indices)
print(test_generator.class_indices)

In [ ]:
def class_weight_computer():
  """
  This function generates the class weights based on training data.
  
    Training Data categories and number of samples in them
    Angry: 3995
    Disgust: 436
    Fear: 4097
    Happy: 7215
    Neutral: 4965
    Sad: 4830
    Surprise: 3171
  """
  samples_per_label = [3995, 436, 4097, 7215, 4965, 4830, 3171]
  total_samples = sum(samples_per_label)
  return dict([(i, total_samples/(7*j)) for (i,j) in enumerate(samples_per_label)]) 

class_weights = class_weight_computer()

### Making Small CNN Network

In [ ]:
DROP_OUT = 0.5
# Building network based on CNN layer only
model = Sequential()

# First convolutional layer
model.add(Conv2D(32, (3, 3), strides=2, input_shape=(img_height, img_width, 3), activation="relu"))
model.add(MaxPool2D((2,2)))
model.add(Dropout(DROP_OUT))

# Second convolutional layer
model.add(Conv2D(64, (3, 3), strides=2, activation="relu"))
model.add(MaxPool2D((2,2)))
model.add(Dropout(DROP_OUT))

# Third convolutional layer
model.add(Conv2D(128, (3, 3), strides=2, activation="relu"))
model.add(MaxPool2D((2,2)))
model.add(Dropout(DROP_OUT))

# Flattening the convolution layers output
model.add(Flatten())

# Fully connected layer
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
# Output layer
model.add(Dense(7, activation='softmax'))

# Compiling the model
model.compile(optimizer=optimizers.Adam(learning_rate=0.001,), loss='categorical_crossentropy', metrics=['accuracy'])
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=0.0001)
model.summary()

In [ ]:
# Model Training
EPOCHS = 50
training_samples = 28709
validation_samples = 3589
test_samples = 3589
history = model.fit_generator(
    train_generator,
    steps_per_epoch=training_samples//batch_size,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=validation_samples//batch_size,
    shuffle=True,
    class_weight=class_weights,
    callbacks=[reduce_lr]
)

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
loss = history.history['loss']

val_acc = history.history['val_accuracy']
val_loss = history.history['val_loss']

epochs = range(1, len(acc)+1)

plt.plot(epochs, acc, 'r', label="Training accuracy")
plt.plot(epochs, val_acc, 'b', label="Validation accuracy")
plt.title('Training and Validation Accuracy')
plt.legend()

plt.figure()
plt.plot(epochs, loss, 'r', label="Training loss")
plt.plot(epochs, val_loss, 'b', label="Validation loss")
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

In [ ]:
print('\nEvaluate on Validation data')
results_validation = model.evaluate_generator(validation_generator, validation_samples//batch_size)
print('Validation loss, Validation Accuracy:', results_validation)

In [ ]:
epoch_str = '-EPOCHS_' + str(EPOCHS)
val_acc = 'val_acc_%.3f' % results_validation[1]

In [ ]:
print(epoch_str)
print(val_acc)

In [ ]:
# Saving the model
model.save("drive/My Drive/ENGR635-Deep Learning System Design Project/Models/Benchmark/" + 'Benchmark' + epoch_str + val_acc + '.h5')

In [ ]:
print('\nEvaluate on test data')
results_test = model.evaluate_generator(test_generator, test_samples//batch_size)
print('test loss, test acc:', results_test)